In [50]:
import pandas as pd

In [135]:
import numpy as np
sales=pd.read_csv('train.csv')

In [176]:
def get_numpy_data(data_frame ,input_feature ,output ):
    data_frame['constant']=1
    input_feature=['constant']+input_feature
    data_frame_sel_feat=data_frame[input_feature]
    data_frame_sel_feat_matrix=data_frame_sel_feat.values
    #output_numarray=data_frame[output].values
    
    return (data_frame_sel_feat_matrix) #,output_numarray)

In [137]:
from sklearn import preprocessing as pl

In [138]:
def normalized_features(feature_matrix):
    norm=np.linalg.norm(feature_matrix)
    feature_matrix=feature_matrix/norm
    return feature_matrix ,norm

In [139]:
features=['LotArea']

In [140]:
test_sales=pd.read_csv('test.csv')

In [188]:
sales['LotArea']=sales['LotArea'].astype(float)
test_sales['LotArea']=test_sales['LotArea'].astype(float)

In [160]:
train_matrix,train_output=get_numpy_data(sales ,features ,'SalePrice')
train_matrix

array([[1.000e+00, 8.450e+03],
       [1.000e+00, 9.600e+03],
       [1.000e+00, 1.125e+04],
       ...,
       [1.000e+00, 9.042e+03],
       [1.000e+00, 9.717e+03],
       [1.000e+00, 9.937e+03]])

In [161]:
def k_nearest_neighbour(feature_matrix ,query_house,k):
    dist2knn=np.empty(k)
    for i in range(k):
        dist2knn[i] =np.sqrt(np.sum((feature_matrix[i]-query_house)**2))
    houses=np.argsort(dist2knn)
    dist2knn.sort()
    for i in range(k ,len(feature_matrix)):
        dist_i=np.sqrt(np.sum((feature_matrix[i]-query_house)**2))
        if(dist_i<dist2knn[k-1]):
            if dist_i<dist2knn[0]:
                dist2knn[1:k]=dist2knn[0:k-1]
                houses[1:k]=houses[0:k-1]
                dist2knn[0]=dist_i
                houses[0]=i
            else:
                for j in range(1 ,k):
                    if dist_i>dist2knn[j-1] and dist_i<dist2knn[j]:
                        dist2knn[j+1:k]=dist2knn[j:k-1]
                        houses[j+1:k]=houses[j:k-1]
                        dist2knn[j]=dist_i
                        houses[j]=i
    
    return(houses)

In [162]:
norm2=np.linalg.norm(train_matrix[:,1], axis=0)

In [163]:
norm2

553927.0577332362

In [164]:
train_matrix[:,1]=train_matrix[:,1]/norm2

In [189]:
test_matrix=get_numpy_data(test_sales ,features ,'SalePrice')
test_matrix[:,1]=test_matrix[:,1]/norm2
test_matrix

array([[1.        , 0.0209811 ],
       [1.        , 0.0257561 ],
       [1.        , 0.02496719],
       ...,
       [1.        , 0.03610584],
       [1.        , 0.01884905],
       [1.        , 0.01737954]])

In [194]:
output_price=[]
sales.loc[sales['Id']==50,'SalePrice'].iloc[0]

127000

In [198]:
for i in range(len(test_matrix)):
    nearest_houses=k_nearest_neighbour(train_matrix ,test_matrix[i:,],6)
    price=0
    for j in range(len(nearest_houses)):
        price+=sales.loc[sales['Id']==nearest_houses[j],'SalePrice'].iloc[0]
        
    price/=6
    output_price.append(price)

In [200]:
len(output_price)

1459

In [208]:
submission=pd.DataFrame(index=None,columns=['Id','SalePrice'])
submission['Id']=test_sales['Id']
submission['SalePrice']=output_price

In [206]:
submission.to_csv('submission1.csv' ,index=False)